In [1]:
import numpy as np
import pandas as pd
import pickle
import json
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
from matplotlib.font_manager import *
myfont = FontProperties(fname='./objs/simhei.ttf', size=14)

In [3]:
def save_obj(obj, name ):
    with open('objs/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('objs/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
ID_city = load_obj('ID_city')
city_ID = load_obj('city_ID')
ID_net = load_obj('ID_net')
city_net = load_obj('city_net')
city_prov = load_obj('city_prov')

In [5]:
len(ID_net)

1122

In [6]:
origin_data = pd.read_csv('data/epi_initial_new.csv')

In [7]:
origin_data = origin_data.sort_values(by = 'ID', ascending= True)

In [8]:
origin_data.head()

,ID,city,population,2020/1/15,2020/1/16,2020/1/17,2020/1/18,2020/1/19,2020/1/20,2020/1/21,2020/1/22,2020/1/23,2020/1/24,2020/1/25,2020/1/26,2020/1/27,2020/1/28,2020/1/29,2020/1/30,2020/1/31
0,0,厦门市,221,0,0,0,0,0,0,0,0,1,3,3,4,4,6,6,13,13
1,1,北京市,1363,0,0,0,0,0,2,5,10,22,36,41,68,80,91,111,132,139
2,2,青岛市,791,0,0,0,0,0,0,1,2,3,4,6,10,13,15,18,21,21
3,3,海口市,167,0,0,0,0,0,0,0,0,2,3,3,3,4,9,9,9,11
4,4,武汉市,834,41,41,45,62,198,198,258,363,433,572,618,698,1590,1905,2261,2639,3215


In [10]:
select_city = ['全国', '北京市', '武汉市', '郑州市', '兰州市', '南昌市', '合肥市', '石家庄市', '天津市', '上海市', '成都市', \
                '广州市', '南京市', '福州市', '乌鲁木齐市', '重庆市', '长沙市', '深圳市', '济南市']

In [11]:
citylist = select_city

In [12]:
population = origin_data[['ID', 'city', 'population']]

In [13]:
epidemic = origin_data.drop(['population', ], axis= 1)

In [14]:
epidemic.shape

(37, 19)

In [15]:
epidemic = np.array(epidemic)

In [18]:
cure_origin = pd.read_csv('data/epi_cure_new.csv')
cure = np.array(cure_origin.drop(['population', ], axis= 1))

In [19]:
death_origin = pd.read_csv('data/epi_death_new.csv')
death = np.array(death_origin.drop(['population', ], axis= 1))

In [20]:
alldates = list(origin_data.columns)[3:]
alldates

['2020/1/15',
 '2020/1/16',
 '2020/1/17',
 '2020/1/18',
 '2020/1/19',
 '2020/1/20',
 '2020/1/21',
 '2020/1/22',
 '2020/1/23',
 '2020/1/24',
 '2020/1/25',
 '2020/1/26',
 '2020/1/27',
 '2020/1/28',
 '2020/1/29',
 '2020/1/30',
 '2020/1/31']

In [21]:
national = epidemic[-1, 2:]
national

array([41, 45, 62, 121, 199, 291, 440, 571, 830, 1274, 1962, 2731, 4502,
       5951, 7698, 9692, 11823], dtype=object)

In [22]:
for city in citylist:
    cityid = city_ID[city]
    print(city, np.sum(epidemic[cityid, 2:] > 0))

全国 17
北京市 12
武汉市 17
郑州市 11
兰州市 9
南昌市 9
合肥市 10
石家庄市 10
天津市 11
上海市 12
成都市 11
广州市 11
南京市 9
福州市 10
乌鲁木齐市 9
重庆市 11
长沙市 11
深圳市 13
济南市 8


In [75]:
cityname= '全国'

In [218]:
cityid = city_ID[cityname]
greater = np.sum(epidemic[cityid, 2:] > 0)
window = max(7, greater)
infect_data = epidemic[cityid, -window:][5:]
cure_data = cure[cityid, -window:][5:]
death_data = death[cityid, -window:][5:]
remove_data = cure_data + death_data
infect_data, remove_data

(array([291, 440, 571, 830, 1274, 1962, 2731, 4502, 5951, 7698, 9692,
        11823], dtype=object),
 array([31, 37, 45, 53, 74, 100, 126, 158, 227, 286, 389, 507],
       dtype=object))

In [219]:
population = np.array(population)
population[-1, 2] = 13000

In [220]:
init_S = population[cityid, 2] * 10000
init_S

130000000

In [27]:
import scipy.integrate as spi
import numpy as np
from sklearn.model_selection import ParameterGrid

def SIR_eqs(INPUT, t, Lambda, beta, mu, gamma):
    Y = np.zeros((3))
    Y[0] = Lambda - ((beta * INPUT[0] * INPUT[1]) / sum(INPUT)) - (mu * INPUT[0])
    Y[1] = ((beta * INPUT[0] * INPUT[1]) / sum(INPUT)) - gamma * INPUT[1] - mu * INPUT[1]
    Y[2] = gamma * INPUT[1] - mu * INPUT[2]

    return Y

def SIR_base_eqs(INPUT, t, beta, gamma):
    Y = np.zeros((3))
    Y[0] = - (beta * INPUT[0] * INPUT[1])
    Y[1] = (beta * INPUT[0] * INPUT[1]) - gamma * INPUT[1]
    Y[2] = gamma * INPUT[1]

    return Y

def SEIR_eqs(INPUT, t, Lambda, beta, mu, epsilon, gamma):
    Y = np.zeros((4))
    Y[0] = Lambda - ((beta * INPUT[0] * INPUT[2]) / sum(INPUT)) - (mu * INPUT[0])
    Y[1] = ((beta * INPUT[0] * INPUT[2]) / sum(INPUT)) - (epsilon * INPUT[1]) - (mu * INPUT[1])
    Y[2] = (epsilon * INPUT[1]) - (gamma * INPUT[2]) - (mu * INPUT[2])
    Y[3] = (gamma * INPUT[2]) - (mu * INPUT[3])

    return Y

def SEIR_base_eqs(INPUT, t, beta, epsilon, gamma):
    Y = np.zeros((4))
    Y[0] = - (beta * INPUT[0] * INPUT[2])
    Y[1] = (beta * INPUT[0] * INPUT[2]) - (epsilon * INPUT[1])
    Y[2] = (epsilon * INPUT[1]) - (gamma * INPUT[2])
    Y[3] = (gamma * INPUT[2])

    return Y

def LSM_loss(TRUE, PRED):
    return np.sum((np.square(TRUE[:, 2 ] - PRED[:, 2 ])))

def GridSearch(ode_eqs, param_grid, param_name, INPUT, t_range, TRUE, loss_func):
    ALL_RES = []
    Grid = ParameterGrid(param_grid)
    for grid in Grid:
        args = [grid[name] for name in param_name]
        PRED = spi.odeint(ode_eqs, INPUT, t_range, args = tuple(args))
        loss = loss_func(TRUE, PRED)
        ALL_RES.append({'loss': loss, 'prediction': PRED, 'parameters': grid})
        
    minloss = ALL_RES[0]['loss']
    minind = 0
    for i, res in enumerate(ALL_RES):
        if res['loss'] < minloss:
            minloss = res['loss']
            minind = i

    return ALL_RES, minind

In [45]:
today = dt.datetime.strptime(alldates[-1], '%Y/%m/%d').date()
dates = [dt.datetime.strptime(d, '%Y/%m/%d').date() for d in alldates[-7:]]
dates += [today + datetime.timedelta(days=x+1) for x in range(3)]

In [33]:
def get_gd(data):
    ind = 20
    last = 0.
    for i in range(ind, len(data) - 2):
        if (data[i] - data[i-1]) / float(data[i-1]) < last:
            return i
        last = (data[i] - data[i-1]) / float(data[i-1])
    return i

## Formal Code

In [34]:
def get_ci(predict_value):
    s = np.random.poisson(predict_value, 1000000)
    return np.percentile(s, [2.5, 97.5])

In [37]:
recover_data = load_obj('recover_data_latest')

In [41]:
np.max(list(recover_data['全国'].keys()))

5

In [97]:
today = dt.datetime.strptime(alldates[-1], '%Y/%m/%d').date()
dates = [dt.datetime.strptime(d, '%Y/%m/%d').date() for d in alldates[-7:]]
dates += [today + datetime.timedelta(days=x+1) for x in range(3)]
if not os.path.exists('./{}_result/'.format(today)):
    os.makedirs('./{}_result/'.format(today))
recover_day = np.max(list(recover_data['全国'].keys())) + 1

def predict_one_city_old(cityname):
    print("Predict city {}".format(cityname))
    cityid = city_ID[cityname]
    greater = np.sum(epidemic[cityid, 2:] > 0)
    window = 8 # max(7, greater)
    infect_data = epidemic[cityid, -window:]
    cure_data = cure[cityid, -window:]
    death_data = death[cityid, -window:]
    remove_data = cure_data + death_data

    print("History Infectious Data ", infect_data)
    
    init_S = population[cityid, 2] * 10000
    print("Initial S = {}".format(init_S))
    
    print("Fitting Model...")
    
    param_grid = {'beta': np.arange(5e-9, 1e-6, 5e-9),
              'gamma': np.arange(1e-3, 0.1, 2e-3), 
              'epsilon': np.arange(0.05, 0.5, 0.05)}
    if cityname == '全国':
        param_grid = {'beta': np.arange(5e-9, 1e-6, 5e-9),
                  'gamma': np.arange(1e-3, 0.1, 2e-3), 
                  'epsilon': np.arange(0.01, 0.1, 0.01)}
    param_name = ['beta', 'gamma', 'epsilon']
    S0 = init_S
    I0 = infect_data[0] - remove_data[0]
    if (I0 == 0):
        I0 = 1.
    E0 = infect_data[0] * 3.
    INPUT = (S0, E0, I0, 0.)
    t_range = np.arange(0.0, len(infect_data), 1.0)
    TRUE = np.random.rand(len(infect_data), 3) * 1e6
    TRUE[:, 2] = infect_data - remove_data
    ALL_RES, minind = GridSearch(SEIR_base_eqs, param_grid, param_name, INPUT, t_range, TRUE, LSM_loss)

    t_range = np.arange(0.0, len(infect_data) + 3, 1.0)
    PRED = spi.odeint(SEIR_base_eqs, INPUT, t_range, tuple([ALL_RES[minind]['parameters'][name] for name in param_name]))

    r0 = ALL_RES[minind]['parameters']['beta'] / ALL_RES[minind]['parameters']['gamma']

    print(ALL_RES[minind]['parameters'])
    print("今天及未来三天预测: ", PRED[:, 2][-4: ])
    print("今天感染: ", infect_data[-1])
        
    plt.plot(dates, PRED[:, 2][-10:], 'b+-')
    plt.plot(dates[:7], infect_data[-7:], "k*:")
    plt.grid("True")
    plt.legend(["Predict Infected", "Official Confirmed Data"])
    plt.title(u'{} 预测结果 (数据截止 {})'.format(cityname, str(today)), FontProperties=myfont)
    plt.xlabel('Date')
    plt.ylabel('Case')
    plt.gcf().autofmt_xdate()
    plt.savefig('./{}_result/predict_{}.jpg'.format(today, cityname))
    plt.show()
    
    t_range = np.arange(0.0, 150, 1.0)
    PRED_ALL = spi.odeint(SEIR_base_eqs, INPUT, t_range, tuple([ALL_RES[minind]['parameters'][name] for name in param_name]))
    
    heap = np.max(PRED_ALL[:, 2])
    gd = get_gd(PRED_ALL[:, 2]) 
    heap_time = np.argmax(PRED_ALL[:, 2]) 
    recover_data[cityname][recover_day] = {'peak': heap, 'inflection': gd, 'peak_left': heap_time}
    
    gd = get_gd(PRED_ALL[:, 2]) - len(infect_data)
    peak_left = np.argmax(PRED_ALL[:, 2]) - len(infect_data)
    print("拐点在 {} 天后，峰值在 {} 天后".format(gd, peak_left))
    print()
    
    predict_data = {}
    predict_data['start_date'] = str(dates[0]); predict_data['end_date'] = str(dates[-1])
    predict_data['value'] = [int(v) for v in PRED[:, 2][-10: ]]
    cilows = []; cihighs = []
    for d, v in zip(dates, list(PRED[:, 2][-10: ])):
        ci_low, ci_high = get_ci(v)
        cilows.append(int(ci_low)); cihighs.append(int(ci_high))
        print('预测值: {:.0f}, 置信区间: [{:.0f}, {:.0f}]'.format(v, ci_low, ci_high))
    predict_data['interval_low'] = cilows; predict_data['interval_high'] = cihighs
    
    official_data = {
        'start_date': str(dates[:7][0]), 'end_date': str(dates[:7][-1]),
        'value': [float(v) for v in infect_data[-7:]]
    }
    
    exp_data = {
        'province': city_prov[cityname],
        'city': cityname,
        'data': {
            'predict': predict_data, 
            'official_confirmed': official_data,
            'peak_left': int(peak_left),
            'inflection_left': int(gd),
#             'recover_coef': recover_data
        }
    }
    
    return {'PRED': PRED[:, 2][-4: ], 'todayinfected': infect_data[-1]}, exp_data

In [ ]:
predictors = {}; jsons = {}
for city in citylist:
    predictors[city], jsons[city] = predict_one_city_old(city)

In [ ]:
# 添加其他城市
other_cities = [x for x in list(origin_data['city']) if not x in citylist]
predictors_other, jsons_other = {}, {}
for city in other_cities:
    predictors_other[city], jsons_other[city] = predict_one_city_old(city)

In [170]:
for city in other_cities:
    predictors[city], jsons[city] = predictors_other[city], jsons_other[city]